## IMPORTING LIBRARIES

In [1]:
import requests,zipfile,io
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import numpy as np
import random
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
import wandb

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [3]:
print(device)

cuda


## DOWNLOADING AND UNZIPPING DATA

In [4]:
def download_data(url="https://drive.google.com/u/0/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download"):
    response=requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall()

## METHODS FOR GETTING CHARACTERS FOR CORPUSS AND ADDING THEIR INDICES

In [5]:
def get_corpus(data):
    eng_corpus=set()
    hin_corpus=set()
    for i in range(0,len(data)):
        eng_word=data[0][i]
        hin_word=data[1][i]
        for ch in eng_word:
            eng_corpus.add(ch)
        for ch in hin_word:
            hin_corpus.add(ch)
        # End Delimiter
        eng_corpus.add('#')
        hin_corpus.add('#')
        hin_corpus.add('$')
        eng_corpus.add('$')
        # Start Delimiter
#         eng_corpus.add('^')
        hin_corpus.add('^')
    return hin_corpus,eng_corpus

In [6]:
def word2index(data):
    hin_corpus,eng_corpus=get_corpus(data)
    engchar_idx={}
    hinchar_idx={}
    idx_engchar={}
    idx_hinchar={}
    i=0
    for char in eng_corpus:
        engchar_idx[char]=i
        idx_engchar[i]=char
        i+=1
    i=0
    for char in hin_corpus:
        hinchar_idx[char]=i
        idx_hinchar[i]=char
        i+=1
    return engchar_idx,hinchar_idx,idx_engchar,idx_hinchar,len(eng_corpus),len(hin_corpus)

## DATA PREPROCESSING

In [7]:
def maxlen(data):
    maxlen_eng=0
    maxlen_hin=0
    for i in range(0,len(data)):
        eng_word=data[0][i]
        hin_word=data[1][i]
        if(len(eng_word)>maxlen_eng):
            maxlen_eng=len(eng_word)
        if(len(hin_word)>maxlen_hin):
            maxlen_hin=len(hin_word)
    return maxlen_eng,maxlen_hin

In [8]:
def pre_process(data,eng_to_idx,hin_to_idx):
    eng=[]
    hin=[]
    maxlen_eng,maxlen_hin=maxlen(data)
    
    unknown= eng_to_idx['$']
    for i in range(0,len(data)):
        sz=0
        eng_word=data[0][i]
        hin_word='^'+data[1][i]
        eng_word = eng_word.ljust(maxlen_eng+1, '#')
        hin_word = hin_word.ljust(maxlen_hin+1, '#')
        idx=[]
        for char in eng_word:
            if eng_to_idx.get(char) is not None:
                idx.append(eng_to_idx[char])
            else:
                idx.append(unknown)
        eng.append(idx)
        idx=[]
        for char in hin_word:
            if hin_to_idx.get(char) is not None:
                idx.append(hin_to_idx[char])
            else:
                idx.append(unknown)
        hin.append(idx)    
    return eng,hin

## LOADING OUR CUSTOM DATASET TO DATALOADER

In [9]:
class MyDataset(Dataset):
    def __init__(self, train_x,train_y, transform=None):
        self.train_x = train_x
        self.train_y = train_y
        self.transform = transform
        
    
    def __len__(self):
        return len(self.train_x)
    
    def __getitem__(self, idx):
        if self.transform:
            sample = self.transform(sample)
        return torch.tensor(self.train_x[idx]).to(device),torch.tensor(self.train_y[idx]).to(device)

def get_data():
    download_data()
    
    train_df=pd.read_csv("aksharantar_sampled/hin/hin_train.csv",header=None)
    test_df=pd.read_csv("aksharantar_sampled/hin/hin_test.csv",header=None)
    val_df=pd.read_csv("aksharantar_sampled/hin/hin_valid.csv",header=None)
    eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=word2index(train_df)
    
    return train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len

## Seq2Seq MODEL

In [10]:
class EncoderGRU(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderGRU,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.gru = nn.GRU(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,hidden=self.gru(embedded,hidden)
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            
        return output,hidden

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderGRU(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.gru = nn.GRU(embedding_size,hidden_size, decoder_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(output))
        return output, hidden

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderRNN,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.rnn = nn.RNN(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,hidden=self.rnn(embedded,hidden)
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            
        return output,hidden

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderRNN(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size, decoder_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output)
        output = self.softmax(self.out(output))
        return output, hidden

In [12]:
class EncoderLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,embedding_size,num_of_layers,batch_size,bi_directional,dropout_p=0.1):
        super(EncoderLSTM,self).__init__()
        self.hidden_size=hidden_size
        self.batch_size=batch_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.embedding=nn.Embedding(input_size,embedding_size)
        self.num_of_layers=num_of_layers
        self.bi_directional=bi_directional
        if(bi_directional=="Yes"):
            flag=True
        else:
            flag=False
        self.lstm = nn.LSTM(embedding_size,hidden_size,num_of_layers,bidirectional=flag)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input,hidden,state):
        embedded=self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,(hidden,state)=self.lstm(embedded,(hidden,state))
    
        if self.bi_directional=="Yes":
            hidden=hidden.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            state=state.resize(2,self.num_of_layers,self.batch_size,self.hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2
            state=torch.add(state[0],hidden[1])/2
            
        return output,hidden,state

    def initHidden(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)
    
    def initState(self):
        if(self.bi_directional=="Yes"):
            return torch.zeros(2*self.num_of_layers,self.batch_size,self.hidden_size,device=device)
        else:
            return torch.zeros(self.num_of_layers,self.batch_size,self.hidden_size,device=device)

class DecoderLSTM(nn.Module):
    def __init__(self, output_size,hidden_size, embedding_size, decoder_layers,batch_size,dropout_p=0.1):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size=embedding_size
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size,hidden_size,decoder_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input,hidden,state):
        embedded = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        output,(hidden,state)=self.lstm(embedded,(hidden,state))
        output = self.out(output)
        return output,hidden,state

## ATTENTION MECHANISM

In [13]:
class AttnDecoder(nn.Module):
    def __init__(self,output_size,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout_p=0.1):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.batch_size=batch_size
        self.cell_type=cell_type
        self.embedding_size=embedding_size
        self.decoder_layers=decoder_layers
        
        self.embedding = nn.Embedding(self.output_size, self.embedding_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.U=nn.Linear(self.hidden_size,self.hidden_size,bias=False).to(device)
        self.W=nn.Linear(self.hidden_size,self.hidden_size,bias=False).to(device)
        self.V=nn.Linear(self.hidden_size,1,bias=False).to(device)
        
        self.linear=nn.Linear(self.hidden_size,output_size,bias=True)
        self.softmax=nn.Softmax()
        if(cell_type=="GRU"):
            self.gru = nn.GRU(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers)
        if(cell_type=="LSTM"):
            self.lstm = nn.LSTM(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers)
        if(cell_type=="RNN"):
            self.rnn = nn.RNN(self.embedding_size+self.hidden_size, self.hidden_size,self.decoder_layers)

    def forward(self, input, hidden,encoder_outputs,word_length,state=None):
        embedded = self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        embedded = self.dropout(embedded)
        T=word_length
        temp=self.W(hidden[-1])
        c=torch.zeros(self.batch_size,self.hidden_size).to(device)
        
        
        for j in range(0,T):
            e_j=self.V(torch.tanh(self.U(encoder_outputs[j])+temp))
            alpha_j=self.softmax(e_j)
            c+=alpha_j*encoder_outputs[j]
        

        final_input=torch.cat((embedded,c.unsqueeze(0)),dim=2)
        
        
        if(self.cell_type=="GRU"):
            output,hidden=self.gru(final_input,hidden)
        if(self.cell_type=="RNN"):
            output,hidden=self.rnn(final_input,hidden)
        if(self.cell_type=="LSTM"):
            output, (hidden,state) =self.lstm(final_input,(hidden,state))
        
        output1=self.linear(output)
        
        if(self.cell_type=="GRU" or self.cell_type=="RNN"):
            return output1, hidden, c
        if(self.cell_type=="LSTM"):
            return output1, hidden, state, c

In [14]:
def train(train_data,encoder,decoder,loss_fun,encoder_optimizer,decoder_optimizer,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention):
    total_loss=0
    teacher_forcing_ratio=0.5
    for i,(train_x,train_y) in enumerate(train_data):
        loss=0
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        train_x=train_x.T
        train_y=train_y.T
        timesteps=len(train_x)
        
        if cell_type=='GRU' or cell_type=='RNN':
            
            encoder_hidden=encoder.initHidden()
            encoder_output,encoder_hidden=encoder(train_x,encoder_hidden)
            if(decoder_layers>encoder_layers):
                i = decoder_layers
                decoder_hidden=encoder_hidden

                while True:
                    if(i==encoder_layers):
                        break
                    # Concatenate the two tensors along the first dimension
                    decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                    i-=1

            elif(decoder_layers<encoder_layers):
                decoder_hidden=encoder_hidden[-decoder_layers:]

            else:
                decoder_hidden=encoder_hidden
        
            decoder_input = train_y[0]
            
            if(bi_directional=="Yes"):
                split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
            
            
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x))
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i] 
                    else:
                        decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i]  # Teacher forcing
            else:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x))
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
                    else:
                        decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            total_loss+=loss
        
        if cell_type=='LSTM':
    
            encoder_hidden=encoder.initHidden()
            encoder_state=encoder.initState()
            
            encoder_output,encoder_hidden,encoder_state=encoder(train_x,encoder_hidden,encoder_state)
        
            if(decoder_layers>encoder_layers):
                i = decoder_layers
                decoder_hidden=encoder_hidden
                decoder_state=encoder_state
                while True:
                    if(i==encoder_layers):
                        break
                    # Concatenate the two tensors along the first dimension
                    decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                    decoder_state = torch.cat([decoder_state, encoder_state[-1].unsqueeze(0)], dim=0)
                    i-=1

            elif(decoder_layers<encoder_layers):
                decoder_hidden=encoder_hidden[-decoder_layers:]
                decoder_state=encoder_state[-decoder_layers:]

            else:
                decoder_hidden=encoder_hidden
                decoder_state=encoder_state
            
            
            if(bi_directional=="Yes"):
                split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
            
            decoder_input = train_y[0]
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            if use_teacher_forcing:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x),decoder_state)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input= train_y[i]
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input = train_y[i]  # Teacher forcing
            else:
                for i in range(0,len(train_y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(train_x),decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), train_y[i])
                        decoder_input=index
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            total_loss+=loss

        
        
    return total_loss.item()/len(train_y),encoder,decoder

In [15]:
def train_iter(input_data,val_data,val_y,input_len,target_len,epochs,batch_size,embedding_size,encoder_layers,decoder_layers,hidden_size,cell_type,bi_directional,dropout,beam_size,attention):
    lr=0.001
    if(cell_type=='GRU'):
        encoder=EncoderGRU(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderGRU(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)
        
    if(cell_type=='RNN'):
        encoder=EncoderRNN(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderRNN(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)
    
    if cell_type=='LSTM':
        encoder=EncoderLSTM(input_len,hidden_size,embedding_size,encoder_layers,batch_size,bi_directional,dropout).to(device)
        if(attention=="Yes"):
            decoder=AttnDecoder(target_len,hidden_size,embedding_size,decoder_layers,batch_size,cell_type,dropout).to(device)
        else:
            decoder=DecoderLSTM(target_len,hidden_size,embedding_size,decoder_layers,batch_size,dropout).to(device)

    encoder_optimizer=optim.Adam(encoder.parameters(),lr)
    decoder_optimizer=optim.Adam(decoder.parameters(),lr)
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    epoch_train_loss=[]
    epoch_val_loss=[]
    epoch_val_acc=[]
    for i in range(0,epochs):
        loss,encoder,decoder=train(input_data,encoder,decoder,loss_fun,encoder_optimizer,decoder_optimizer,
                                   encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,
                                   cell_type,attention)
        val_predictions,val_loss=eval(val_data,encoder,decoder,encoder_layers,decoder_layers,
                                  batch_size,hidden_size,bi_directional,cell_type,attention)
        
        epoch_val_loss.append(val_loss)
        epoch_train_loss.append(loss/51200)
        
        val_acc=accuracy(val_predictions,val_y)
        epoch_val_acc.append(val_acc)
        print(loss/51200,val_loss,val_acc)
    
#     train_predictions,t=eval(input_data,encoder,decoder,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention)
    return epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers

In [16]:
def eval(input_data,encoder,decoder,encoder_layers,decoder_layers,batch_size,hidden_size,bi_directional,cell_type,attention):
    with torch.no_grad():
        loss_fun=nn.CrossEntropyLoss(reduction="sum")
        total_loss=0
        pred_words=list()
        for x,y in input_data:
            loss=0
            decoder_words=[]
            x=x.T
            y=y.T
            encoder_hidden=encoder.initHidden()
            timesteps=len(x)
            if cell_type=='GRU' or cell_type=='RNN':

                encoder_hidden=encoder.initHidden()
                encoder_output,encoder_hidden=encoder(x,encoder_hidden)
                if(decoder_layers>encoder_layers):
                    i = decoder_layers
                    decoder_hidden=encoder_hidden

                    while True:
                        if(i==encoder_layers):
                            break
                        # Concatenate the two tensors along the first dimension
                        decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                        i-=1

                elif(decoder_layers<encoder_layers):
                    decoder_hidden=encoder_hidden[-decoder_layers:]

                else:
                    decoder_hidden=encoder_hidden

                decoder_input = y[0]

                if(bi_directional=="Yes"):
                    split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                    encoder_output=torch.add(split_tensor[0],split_tensor[1])/2

                for i in range(0,len(y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(x))
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                    else:
                        decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                decoder_words=np.array(decoder_words)
                pred_words.append(decoder_words.T)
                total_loss+=loss.item()


            if cell_type=='LSTM':

                encoder_hidden=encoder.initHidden()
                encoder_state=encoder.initState()

                encoder_output,encoder_hidden,encoder_state=encoder(x,encoder_hidden,encoder_state)

                if(decoder_layers>encoder_layers):
                    i = decoder_layers
                    decoder_hidden=encoder_hidden
                    decoder_state=encoder_state
                    while True:
                        if(i==encoder_layers):
                            break
                        # Concatenate the two tensors along the first dimension
                        decoder_hidden = torch.cat([decoder_hidden, encoder_hidden[-1].unsqueeze(0)], dim=0)
                        decoder_state = torch.cat([decoder_state, encoder_state[-1].unsqueeze(0)], dim=0)
                        i-=1

                elif(decoder_layers<encoder_layers):
                    decoder_hidden=encoder_hidden[-decoder_layers:]
                    decoder_state=encoder_state[-decoder_layers:]

                else:
                    decoder_hidden=encoder_hidden
                    decoder_state=encoder_state


                if(bi_directional=="Yes"):
                    split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
                    encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
                decoder_input = y[0]

                for i in range(0,len(y)):
                    if(attention=="Yes"):
                        decoder_output, decoder_hidden, decoder_state, attn_weights=decoder(decoder_input,decoder_hidden,encoder_output,len(x),decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                    else:
                        decoder_output, decoder_hidden,decoder_state= decoder(decoder_input, decoder_hidden,decoder_state)
                        max_prob,index=decoder_output.topk(1)
                        loss+=loss_fun(torch.squeeze(decoder_output), y[i])
                        index=index.squeeze()
                        decoder_input=index
                        decoder_words.append(index.tolist())
                decoder_words=np.array(decoder_words)
                pred_words.append(decoder_words.T)
                total_loss+=loss.item()


    predictions=[]
    for batch in pred_words:
        for word in batch:
            predictions.append(word)

    return predictions,total_loss/(len(predictions)*len(predictions[0]))

In [17]:
def accuracy(predictions,y):
    count=0
#     print(len(y),len(y[0]))
    for i in range(0,len(predictions)):
        p=predictions[i]
        if np.array_equal(p,y[i]):
            count+=1
    return (count/len(predictions))*100

## MODEL

In [18]:
# epochs=20
# batchsize=64
# embedding_size=256
# encoder_layers=2
# decoder_layers=3
# hidden_size=256
# cell_type="LSTM"
# bi_directional="Yes"
# dropout=0.2
# beam_size=16
# attention="No"

In [19]:
# train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=get_data()

# train_x,train_y = pre_process(train_df,eng_to_idx,hin_to_idx)
# test_x,test_y = pre_process(test_df,eng_to_idx,hin_to_idx)
# val_x,val_y = pre_process(val_df,eng_to_idx,hin_to_idx)

# train_dataset=MyDataset(train_x,train_y)
# test_dataset=MyDataset(test_x,test_y)
# val_dataset=MyDataset(val_x,val_y)


# train_dataloader=DataLoader(train_dataset,batch_size=batchsize)
# test_dataloader=DataLoader(test_dataset,batch_size=batchsize)
# val_dataloader=DataLoader(val_dataset,batch_size=batchsize)

In [20]:
# epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers=train_iter(train_dataloader,val_dataloader,val_y,input_len,target_len,epochs,batchsize,embedding_size,encoder_layers,decoder_layers,hidden_size,cell_type,bi_directional,dropout,beam_size,attention)

In [21]:
# print(epoch_val_acc)

In [22]:
# test_predictions,loss=eval(test_dataloader,encoder,decoder,encoder_layers,decoder_layers,batchsize,hidden_size,bi_directional,cell_type,attention)

In [23]:
# test_accuracy=accuracy(test_predictions,test_y)

In [24]:
# print(test_accuracy)

## INTEGRATING WITH WANDB

In [25]:
def wandb_run_sweeps(train_dataset,val_dataset,test_dataset,train_y,val_y,test_y,input_len,target_len):
    
    config = {
        "project":"CS6910_Assignment3",
        "method": 'random',
        "metric": {
        'name': 'acc',
        'goal': 'maximize'
        },
        'parameters' :{
        "epochs": {"values":[10,20]},
        "batchsize": {"values": [64,128,256]},
        "embedding_size": {"values":[16, 32, 64, 256, 512]},
        "hidden_size": {"values":[16, 32, 64, 256, 512]},
        "encoder_layers": {"values":[2,3,4]},
        "decoder_layers": {"values":[2,3,4]},
        "cell_type": {"values":["RNN","GRU","LSTM"]},
        "bi_directional":{"values":["Yes","No"]},
        "dropout":{"values":[0.1,0.2,0.5]},
        "attention":{"values":["Yes","No"]},
        "beam_size":{"values":[5,10,15]}
        }
    }
    def train_rnn():
        wandb.init()

        name='_CT_'+str(wandb.config.cell_type)+"_BS_"+str(wandb.config.batchsize)+"_EPOCH_"+str(wandb.config.epochs)+"_ES_"+str(wandb.config.embedding_size)+"_HS_"+str(wandb.config.hidden_size)
        
        
        train_dataloader=DataLoader(train_dataset,batch_size=wandb.config.batchsize)
        test_dataloader=DataLoader(test_dataset,batch_size=wandb.config.batchsize)
        val_dataloader=DataLoader(val_dataset,batch_size=wandb.config.batchsize)
        
        epoch_train_loss,epoch_val_loss,epoch_val_acc,encoder,decoder,encoder_layers,decoder_layers=train_iter(train_dataloader,val_dataloader,val_y,input_len,target_len,wandb.config.epochs,wandb.config.batchsize,wandb.config.embedding_size,wandb.config.encoder_layers,wandb.config.decoder_layers,wandb.config.hidden_size,wandb.config.cell_type,wandb.config.bi_directional,wandb.config.dropout,wandb.config.beam_size,wandb.config.attention)

        for i in range(wandb.config.epochs):
            wandb.log({"loss":epoch_train_loss[i]})
            wandb.log({"val_loss":epoch_val_loss[i]})
            wandb.log({"val_acc":epoch_val_acc[i]})
            wandb.log({"epoch": (i+1)})
        wandb.log({"validation_accuracy":epoch_val_acc[-1]})    
        
        train_predictions,_=eval(train_dataloader,encoder,decoder,wandb.config.encoder_layers,
                              wandb.config.decoder_layers,wandb.config.batchsize,wandb.config.hidden_size,
                              wandb.config.bi_directional,wandb.config.cell_type,wandb.config.attention)

        train_accuracy=accuracy(train_predictions,train_y)
        wandb.log({"train_accuracy":train_accuracy})
        
        test_predictions,_=eval(test_dataloader,encoder,decoder,wandb.config.encoder_layers,
                              wandb.config.decoder_layers,wandb.config.batchsize,wandb.config.hidden_size,
                              wandb.config.bi_directional,wandb.config.cell_type,wandb.config.attention)

        test_accuracy=accuracy(test_predictions,test_y)
        wandb.log({"test_accuracy":test_accuracy})
        wandb.run.name = name
        wandb.run.save()
        wandb.run.finish()
    wandb.login(key="aecb4b665a37b40204530b0627a42274aeddd3e1")
    sweep_id=wandb.sweep(config,project="CS6910_Assignment3")
    wandb.agent(sweep_id,function=train_rnn,count=20)

In [26]:
train_df,test_df,val_df,eng_to_idx,hin_to_idx,idx_to_eng,idx_to_hin,input_len,target_len=get_data()

train_x,train_y = pre_process(train_df,eng_to_idx,hin_to_idx)
test_x,test_y = pre_process(test_df,eng_to_idx,hin_to_idx)
val_x,val_y = pre_process(val_df,eng_to_idx,hin_to_idx)

train_dataset=MyDataset(train_x,train_y)
test_dataset=MyDataset(test_x,test_y)
val_dataset=MyDataset(val_x,val_y)

wandb_run_sweeps(train_dataset,val_dataset,test_dataset,train_y,val_y,test_y,input_len,target_len)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: qq1xrvam
Sweep URL: https://wandb.ai/cs22m078/CS6910_Assignment3/sweeps/qq1xrvam


wandb: Agent Starting Run: pnlx7nyv with config:
wandb: 	attention: No
wandb: 	batchsize: 256
wandb: 	beam_size: 10
wandb: 	bi_directional: No
wandb: 	cell_type: GRU
wandb: 	decoder_layers: 2
wandb: 	dropout: 0.2
wandb: 	embedding_size: 512
wandb: 	encoder_layers: 3
wandb: 	epochs: 10
wandb: 	hidden_size: 16
wandb: Currently logged in as: cs22m078. Use `wandb login --relogin` to force relogin


3.6452071707589284 2.266708805447533 0.0
2.140413643973214 1.641264211563837 0.0
2.029067847842262 1.5635643005371094 0.0
1.872418271019345 1.453818440437317 0.0
1.8069174339657739 1.4316247190747942 0.0
1.722648460751488 1.485798642748878 0.0
1.7285330636160714 1.4132286707560222 0.0
1.6430598958333333 1.3880329699743361 0.0
1.5322760881696427 1.3602327676046462 0.0
1.54264892578125 1.3991765124457223 0.0


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

1.5222963169642858 1.149841437737147 0.0
1.2971844773065475 1.1056677003701527 0.0
1.2147366768973213 1.0642197075344266 0.0
1.1519305710565475 1.0085400740305583 0.0244140625
1.0967921084449406 0.9481095529737926 0.0732421875
1.0468924386160714 0.9043796105044228 0.244140625
1.003049083891369 0.8903561333815256 0.2197265625
0.9712829008556548 0.8651931151038125 0.341796875
0.9403611537388393 0.848476672456378 0.341796875
0.9136065383184524 0.8363199659756252 0.5615234375
0.8798432849702381 0.8006411875997271 0.830078125
0.8630818684895832 0.7773186351571765 0.9521484375
0.8363820684523811 0.7608973128455025 1.5869140625
0.821109386625744 0.7632800965082078 1.9287109375
0.7972320847284226 0.7462748892250515 2.1728515625
0.778038562593006 0.7417981603315899 2.0263671875
0.7647118559337798 0.7247082662014734 3.0029296875
0.7467515345982143 0.7236880972271874 3.076171875
0.7280057198660714 0.7036481166169757 3.6865234375
0.7243287876674107 0.6901430913380214 3.02734375


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
test_accuracy,▁
train_accuracy,▁
val_acc,▁▁▁▁▁▁▁▂▂▂▃▃▄▅▅▅▇▇█▇
val_loss,█▇▇▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁
validation_accuracy,▁
epoch,20
loss,0.72433
test_accuracy,3.24707
train_accuracy,1.85352


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ygux4yzd with config:
wandb: 	attention: Yes
wandb: 	batchsize: 128
wandb: 	beam_size: 5
wandb: 	bi_directional: Yes
wandb: 	cell_type: LSTM
wandb: 	decoder_layers: 4
wandb: 	dropout: 0.1
wandb: 	embedding_size: 32
wandb: 	encoder_layers: 2
wandb: 	epochs: 20
wandb: 	hidden_size: 16


1.9614734468005952 1.2238169766607738 0.0
1.374841076078869 1.1583850270225888 0.0
1.309494861421131 1.1048751473426819 0.0
1.2534951636904763 1.05692210935411 0.0
1.2054427083333332 1.0003062401499068 0.0
1.1623709542410714 0.9755965982164655 0.0
1.1213631184895834 0.9357057639530727 0.0244140625
1.085332496279762 0.9077214513506208 0.0244140625
1.0500760323660714 0.8912830324400038 0.0732421875
1.0183858816964286 0.8699745280402047 0.1220703125
0.99902099609375 0.8664898106030056 0.1708984375
0.9832378859747024 0.8349934617678324 0.244140625
0.9630300176711311 0.8262676823706854 0.3173828125
0.9443851143973214 0.8046604664552779 0.48828125
0.9281647019159226 0.7960791871661231 0.48828125
0.9113262067522321 0.7917549524988446 0.3662109375
0.8997060430617561 0.7799451663380578 0.439453125
0.8831096540178571 0.7752973706949324 0.732421875
0.8771052478608631 0.7611341958954221 0.87890625
0.86157470703125 0.7532032146340325 0.68359375


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
test_accuracy,▁
train_accuracy,▁
val_acc,▁▁▁▁▁▁▁▁▂▂▂▃▄▅▅▄▅▇█▆
val_loss,█▇▆▆▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
validation_accuracy,▁
epoch,20
loss,0.86157
test_accuracy,0.83008
train_accuracy,0.32617
